# About this notebook:
- We will be using articles IDs to collect each articles' information, including:
    - Title
    - Abstract
    - Given Mesh Terms
    - Publication Date
- This notebook is part of a series of notebooks under 'Step 2 Articles Collection', where each notebook contain articles published in a particular year.

# Import Libraries and Articles' IDs

In [1]:
# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from Bio import Entrez
import xml.etree.ElementTree as ET

import time
from tqdm import tqdm

# Set the email address associated with your NCBI account
Entrez.email = "geok1723@gmail.com"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df=pd.read_csv("../Step_1_ID_Collection/IDs_2017.csv")

In [2]:
df.head()

,Date_str,ID,Date,Month,Year
0,2017/01/31,28131197,2017-01-31,1,2017
1,2017/01/31,28134633,2017-01-31,1,2017
2,2017/01/24,28111826,2017-01-24,1,2017
3,2017/01/25,28115833,2017-01-25,1,2017
4,2017/01/26,28121180,2017-01-26,1,2017


# Collecting Articles in groups of 100<br>
> The IDs are split into mini-dataframes containing 100 IDs for scraping. <br>
> Output will contain about 360 mini dataframes of articles that are published in this particular year. <br>
> This process was executed using Kaggle, and resulting dataframes are stored in the respective zip-folder. <br>

In [3]:
# determine the group number for each row
group_num = (df.index // 100) + 1

# map the group number to a group label (G1 to G6)
group_label = 'G' + group_num.astype(str)

# add the new column to the DataFrame
df['sub_group'] = group_label

# print(df)
# print(df['sub_group'].nunique())

df['Date']=pd.to_datetime(df['Date'],yearfirst=True)

# group the DataFrame by "Sub_group"
grouped = df.groupby('sub_group')

# create an empty dictionary to store the smaller DataFrames
df_dict = {}

# loop over the groups and create smaller DataFrames
for subgroup, subgroup_df in grouped:
    # construct the name of the smaller DataFrame
    year = subgroup_df['Year'].iloc[0]
    month = subgroup_df['Date'].iloc[0].strftime('%m')
    df_name = f"df_{year}_{month}_{subgroup}"
    # add the smaller DataFrame to the dictionary
    df_dict[df_name] = subgroup_df.copy()

df_ls = list(df_dict.keys())

df_ls.sort()

In [7]:
output_df = {}
for grp in tqdm(df_ls):
    _df = df_dict[grp]
    _s = _df[['ID','Date_str']] 

    print(f"Creating Dataframe of articles for {grp}")
    abstracts = []
    titles = []
    ids = []
    MeSH = []
    Pub_Date = []

    for index,row in tqdm(_s.iterrows()):	

        pub_date = row['Date_str']
        pmid = row['ID']


        try:
            handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="xml")

            xml_data = handle.read()
            handle.close()

            #obtain abstract and title
            root = ET.fromstring(xml_data)
            title = root.find(".//ArticleTitle").text.strip()
            abst = root.findall(".//AbstractText")

            abstract_text = ""
            for abstract in abst:
                abstract_text += abstract.text

            
            #obtain MeSH
            mesh_terms = ['']
            mesh_list = root.findall(".//MeshHeadingList/MeshHeading")
            for mesh in mesh_list:
                descriptor_name = mesh.find(".//DescriptorName").text
                qualifiers = mesh.findall(".//QualifierName")
                qualifier_names = [qualifier.text for qualifier in qualifiers]
                mesh_terms.append((descriptor_name, qualifier_names))
                
            MeSH.append(mesh_terms)            
            ids.append(pmid)
            abstracts.append(abstract_text)
            titles.append(title)
            Pub_Date.append(pub_date)

        except:
            print(f"Error here in ID{pmid}")

    # Store the data in a pandas dataframe
    data = {"ID": ids, "title": titles,"Pub_Date":Pub_Date, "abstract": abstracts, "MeSH_term":MeSH}
    data_df = pd.DataFrame(data)
    df_name = f"ABSTRACT{grp}"
    output_df[df_name] = data_df
    data_df.to_csv(f"{df_name}.csv", index=False)
    print(output_df.keys())

  0%|          | 0/360 [00:00<?, ?it/s]

Creating Dataframe of articles for df_2017_01_G1



0it [00:00, ?it/s]
1it [00:00,  2.08it/s]
2it [00:00,  2.05it/s]
3it [00:01,  1.93it/s]
4it [00:02,  1.95it/s]
5it [00:02,  2.07it/s]
6it [00:02,  2.04it/s]
7it [00:03,  2.14it/s]
8it [00:03,  2.20it/s]
9it [00:04,  2.25it/s]
10it [00:04,  2.16it/s]
11it [00:05,  2.16it/s]
12it [00:05,  2.21it/s]
13it [00:06,  2.25it/s]
14it [00:06,  2.09it/s]
15it [00:07,  1.96it/s]
16it [00:07,  2.04it/s]
17it [00:08,  2.10it/s]
18it [00:08,  2.08it/s]
19it [00:08,  2.16it/s]
20it [00:09,  2.11it/s]
21it [00:09,  2.08it/s]

Error here in ID28042158



22it [00:10,  1.98it/s]
23it [00:10,  2.07it/s]
24it [00:11,  2.15it/s]
25it [00:11,  2.05it/s]
26it [00:12,  2.13it/s]
27it [00:12,  2.18it/s]
28it [00:13,  2.22it/s]
29it [00:13,  2.00it/s]
30it [00:14,  2.08it/s]
31it [00:14,  2.16it/s]
32it [00:15,  2.12it/s]
33it [00:15,  1.99it/s]
34it [00:16,  2.09it/s]
35it [00:16,  2.15it/s]
36it [00:17,  2.12it/s]
37it [00:17,  2.10it/s]
38it [00:18,  1.98it/s]
39it [00:19,  1.38it/s]
40it [00:19,  1.53it/s]
41it [00:20,  1.55it/s]
42it [00:21,  1.67it/s]
43it [00:21,  1.76it/s]
44it [00:21,  1.91it/s]
45it [00:22,  1.95it/s]
46it [00:22,  2.05it/s]
47it [00:23,  2.14it/s]
48it [00:23,  2.08it/s]
49it [00:24,  2.07it/s]
50it [00:24,  2.05it/s]
51it [00:25,  2.14it/s]
52it [00:25,  2.19it/s]
53it [00:26,  2.14it/s]
54it [00:26,  2.08it/s]
55it [00:27,  2.07it/s]
56it [00:27,  2.16it/s]
57it [00:28,  2.12it/s]
58it [00:28,  2.10it/s]
59it [00:29,  1.99it/s]
60it [00:29,  1.99it/s]
61it [00:30,  2.09it/s]
62it [00:30,  2.07it/s]
63it [00:30,  2

...

Error here in ID29209420



94it [00:45,  1.43it/s]
95it [00:46,  1.50it/s]
96it [00:46,  1.68it/s]
97it [00:46,  1.84it/s]
98it [00:47,  1.98it/s]
99it [00:47,  2.08it/s]
100it [00:48,  2.07it/s]
100%|█████████▉| 359/360 [5:07:00<00:48, 48.13s/it]

dict_keys(['ABSTRACTdf_2017_01_G1', 'ABSTRACTdf_2017_01_G10', 'ABSTRACTdf_2017_01_G11', 'ABSTRACTdf_2017_01_G12', 'ABSTRACTdf_2017_01_G13', 'ABSTRACTdf_2017_01_G14', 'ABSTRACTdf_2017_01_G15', 'ABSTRACTdf_2017_01_G16', 'ABSTRACTdf_2017_01_G17', 'ABSTRACTdf_2017_01_G18', 'ABSTRACTdf_2017_01_G19', 'ABSTRACTdf_2017_01_G2', 'ABSTRACTdf_2017_01_G20', 'ABSTRACTdf_2017_01_G21', 'ABSTRACTdf_2017_01_G22', 'ABSTRACTdf_2017_01_G23', 'ABSTRACTdf_2017_01_G24', 'ABSTRACTdf_2017_01_G25', 'ABSTRACTdf_2017_01_G26', 'ABSTRACTdf_2017_01_G27', 'ABSTRACTdf_2017_01_G28', 'ABSTRACTdf_2017_01_G29', 'ABSTRACTdf_2017_01_G3', 'ABSTRACTdf_2017_01_G30', 'ABSTRACTdf_2017_01_G4', 'ABSTRACTdf_2017_01_G5', 'ABSTRACTdf_2017_01_G6', 'ABSTRACTdf_2017_01_G7', 'ABSTRACTdf_2017_01_G8', 'ABSTRACTdf_2017_01_G9', 'ABSTRACTdf_2017_02_G31', 'ABSTRACTdf_2017_02_G32', 'ABSTRACTdf_2017_02_G33', 'ABSTRACTdf_2017_02_G34', 'ABSTRACTdf_2017_02_G35', 'ABSTRACTdf_2017_02_G36', 'ABSTRACTdf_2017_02_G37', 'ABSTRACTdf_2017_02_G38', 'ABSTRACTd


0it [00:00, ?it/s]
1it [00:00,  1.97it/s]
2it [00:01,  1.96it/s]
3it [00:01,  2.12it/s]
4it [00:01,  2.08it/s]
5it [00:02,  2.17it/s]
6it [00:02,  2.21it/s]
7it [00:03,  2.15it/s]
8it [00:03,  2.21it/s]
9it [00:04,  2.27it/s]
10it [00:04,  2.20it/s]
11it [00:05,  2.26it/s]
12it [00:05,  2.07it/s]
13it [00:06,  2.04it/s]
14it [00:06,  2.01it/s]
15it [00:07,  2.01it/s]
16it [00:07,  2.08it/s]
17it [00:07,  2.17it/s]
18it [00:08,  1.96it/s]
19it [00:09,  1.80it/s]
20it [00:09,  1.79it/s]
21it [00:10,  1.86it/s]
22it [00:10,  1.98it/s]
23it [00:11,  2.09it/s]
24it [00:11,  2.02it/s]
25it [00:12,  2.01it/s]
26it [00:12,  2.02it/s]
27it [00:13,  2.11it/s]
28it [00:13,  2.00it/s]
29it [00:14,  2.08it/s]
30it [00:14,  2.15it/s]
31it [00:15,  2.08it/s]
32it [00:15,  2.07it/s]
33it [00:15,  2.16it/s]
34it [00:16,  1.99it/s]
35it [00:16,  2.10it/s]
36it [00:17,  2.07it/s]
37it [00:17,  2.06it/s]
38it [00:18,  2.15it/s]
39it [00:18,  2.21it/s]
40it [00:19,  2.27it/s]
41it [00:19,  2.29it/s]
42it 

dict_keys(['ABSTRACTdf_2017_01_G1', 'ABSTRACTdf_2017_01_G10', 'ABSTRACTdf_2017_01_G11', 'ABSTRACTdf_2017_01_G12', 'ABSTRACTdf_2017_01_G13', 'ABSTRACTdf_2017_01_G14', 'ABSTRACTdf_2017_01_G15', 'ABSTRACTdf_2017_01_G16', 'ABSTRACTdf_2017_01_G17', 'ABSTRACTdf_2017_01_G18', 'ABSTRACTdf_2017_01_G19', 'ABSTRACTdf_2017_01_G2', 'ABSTRACTdf_2017_01_G20', 'ABSTRACTdf_2017_01_G21', 'ABSTRACTdf_2017_01_G22', 'ABSTRACTdf_2017_01_G23', 'ABSTRACTdf_2017_01_G24', 'ABSTRACTdf_2017_01_G25', 'ABSTRACTdf_2017_01_G26', 'ABSTRACTdf_2017_01_G27', 'ABSTRACTdf_2017_01_G28', 'ABSTRACTdf_2017_01_G29', 'ABSTRACTdf_2017_01_G3', 'ABSTRACTdf_2017_01_G30', 'ABSTRACTdf_2017_01_G4', 'ABSTRACTdf_2017_01_G5', 'ABSTRACTdf_2017_01_G6', 'ABSTRACTdf_2017_01_G7', 'ABSTRACTdf_2017_01_G8', 'ABSTRACTdf_2017_01_G9', 'ABSTRACTdf_2017_02_G31', 'ABSTRACTdf_2017_02_G32', 'ABSTRACTdf_2017_02_G33', 'ABSTRACTdf_2017_02_G34', 'ABSTRACTdf_2017_02_G35', 'ABSTRACTdf_2017_02_G36', 'ABSTRACTdf_2017_02_G37', 'ABSTRACTdf_2017_02_G38', 'ABSTRACTd